In [25]:
import pandas as pd
import numpy as np


# Given the explanation of the header, we will define a function to parse it and extract the relevant information.
def parse_header(header):
    # Splitting the header string by underscores
    parts = header.split('_')
    
    # Extracting the information based on the index
    architectural_archetype = parts[0]  # C
    stories = int(parts[1])            # 3
    soil_class = parts[4]              # D
    seismic_zone = int(parts[6])       # 3
    connection_system = parts[8]       # HD
    
    # Creating a dictionary to hold the extracted information
    extracted_info = {
        "architectural_archetype": architectural_archetype,
        "stories": stories,
        "soil_class": soil_class,
        "seismic_zone": seismic_zone,
        "connection_system": connection_system
    }
    
    return extracted_info


# We have an issue where in an excel file if a column is represented by multiple rows, the first one will have the correct value and then the others will have nan values
def fill_values_based_on_key(data, key_column_index, value_row_index, finishing_row_informationB):
    """
    Fill the values in the value column based on the last non-NaN value in the key column.

    :param data: The 2D list (list of lists) representing the data.
    :param key_column_index: The index of the column to use as the key.
    :param value_row_index: The index of the row where values are to be filled.
    :return: None; the operation modifies the data list in place.
    """
    last_valid_key = None
    for i in range(value_row_index, finishing_row_informationB):
        key_value = data.iat[i, key_column_index]
        if pd.notna(key_value):  # Check if the key column value is not NaN
            last_valid_key = key_value
        if pd.isna(key_value):  # Check if the value column is NaN
            data.iat[i, key_column_index] = last_valid_key



def prepare_data_to_csv(file_path, sheet_name) :
    
    # Load the Excel file
    # For this file we will read it with a header since it is easier to extract the information of type A
    data = pd.read_excel(file_path, sheet_name=sheet_name)
    
    # To find the number of buildings, we need to find how many unamed columns there are
    nbr_building = len([col for col in data.columns if not 'Unnamed' in str(col)])

    #general parameters to understand better to not have random parameters
    nbr_story = 5 #this parameter is not used in my code but maybe in another
    starting_row_informationB = 14 #normally same for all files
    size_columns_informationA = 5 #normally same for all files
    row_Tx_Ty_values = 12


    # Extract Type A Information

    # Step 1: Parsing headers and store the results
    parsed_data = []
    for i in range(1, nbr_building+1):
        header = data[i][1]
        parsed_data.append(parse_header(header))

    # Step 2: Defining the desired columns
    columns = ["architectural_archetype", "stories", "soil_class", "seismic_zone"]

    # Step 3: Create a new table and populate it with the parsed data
    new_table = []
    for item in parsed_data:
        row = [item[col] for col in columns]
        new_table.append(row)

    # 'new_table' now contains the parsed data structured as rows and columns
    df = pd.DataFrame(new_table, columns=columns)


    data2 = pd.read_excel(file_path,sheet_name=sheet_name, header=None)
    
    # We want to find the finishing row of the tables (we don't know sine it is an excel file without headers)
    # We want to consider rows that have at least one non-NaN value:
    finishing_row_informationB = data.dropna(how='all').index[-1] + 1


    # Assuming data2 is a list of lists representation of your Excel data
    fill_values_based_on_key(data2, 3, starting_row_informationB, finishing_row_informationB)
    fill_values_based_on_key(data2, 4, starting_row_informationB, finishing_row_informationB)


    repetitions = finishing_row_informationB - starting_row_informationB

    #Add type A information
    # Repeat each row in the DataFrame
    repeated_df = pd.DataFrame(np.repeat(df.values, repetitions, axis=0), columns=df.columns)

    #Add type B information part 1
    # Replicate df1 twelve times, ensuring the column names are as expected.
    df1 = [data2.iloc[starting_row_informationB:finishing_row_informationB, 3:8]
           .rename(columns={data2.columns[3]: "Story",
                                                    data2.columns[4]: "Direction",
                                                    data2.columns[5]: "Wall",
                                                    data2.columns[6]: "L [cm]",
                                                    data2.columns[7]: "xi [cm]"}) for _ in range(nbr_building)]
    #Add type C information
    # Create subsets using list comprehensions for dfs with the correct column names
    dfs = [data2.iloc[starting_row_informationB:finishing_row_informationB,
                      9 + size_columns_informationA * i : 13 + size_columns_informationA * i]
           .rename(columns={data2.columns[9 + size_columns_informationA * i]: "Nail spacing [cm]",
                   data2.columns[10 + size_columns_informationA * i]: "Number sheathing panels",
                   data2.columns[11 + size_columns_informationA * i]: "Number end studs",
                   data2.columns[12 + size_columns_informationA * i]: "Total number studs"}) for i in range(nbr_building)]


    # Concatenate lists of DataFrames, ensuring the column names align.
    result2 = pd.concat(df1, ignore_index=True)
    result3 = pd.concat(dfs, ignore_index=True)


    #We have to add additional type B information (part 2)
    #We will also add additional type C information (part2)
    # Initialize empty lists to store the D+0.25L and Story Area values
    d_plus_quarter_l_values = []
    story_area_values = []
    Tx_values = []
    Ty_values = []

    # Loop through the specified ranges and compute the values
    for i in range(0, nbr_building):
        for j in range(0, finishing_row_informationB-starting_row_informationB):
            # Extract D+0.25L value and append to the list 
            story = int(result2.iat[j, 0])

            d_plus_quarter_l =  data2.iat[4 + story, 11 + size_columns_informationA * i]
            d_plus_quarter_l_values.append(d_plus_quarter_l)

            # Extract Story Area value and append to the list
            story_area = data2.iat[4 + story, 13 + size_columns_informationA * i]
            story_area_values.append(story_area)

            Tx_values.append(data2.iat[row_Tx_Ty_values, 9 + size_columns_informationA * i])
            Ty_values.append(data2.iat[row_Tx_Ty_values, 10 + size_columns_informationA * i])


    # Add the new columns to result2
    result2['D+0.25L'] = d_plus_quarter_l_values
    result2['Story Area'] = story_area_values
    result3['Tx(s)'] = Tx_values
    result3['Ty(s)'] = Ty_values

    # Concatenate horizontally, ensuring that both dataframes have the same index
    resultFinal = pd.concat([repeated_df, result2, result3], axis=1, ignore_index=False)
    
    
    #Creating another table if we are using only information of type A to predict Tx and Ty not everything
    Tx_values = []
    Ty_values = []

    # Loop through the specified ranges and compute the values
    for i in range(0, nbr_building): 
            Tx_values.append(data2.iat[row_Tx_Ty_values, 9 + size_columns_informationA * i])
            Ty_values.append(data2.iat[row_Tx_Ty_values, 10 + size_columns_informationA * i])

    df['Tx(s)'] = Tx_values
    df['Ty(s)'] = Ty_values

    return resultFinal, df


# For each file, file_path sheet (page of the excel file)
# List of Excel files
files = ['./Design_P_ATS.xlsx', './Design_P_HD.xlsx',
         './Design_D_ATS.xlsx', './Design_D_HD.xlsx',
         './Design_C_ATS.xlsx', './Design_C_HD.xlsx',
         './Design_Q_ATS.xlsx', './Design_Q_HD.xlsx']

# Initialize an empty list to store file and sheet index information
Files_informations = []

# For each file, get all sheet names and add to Files_informations
for file in files:
    xls = pd.ExcelFile(file)
    sheet_names = xls.sheet_names
    for sheet_index, sheet_name in enumerate(sheet_names):
        Files_informations.append([file, sheet_index])

resultsFinal = []
resultsDF = []
for i in range(len(Files_informations)):
    resultFinal, df = prepare_data_to_csv(Files_informations[i][0], Files_informations[i][1])
    resultsFinal.append(resultFinal)
    resultsDF.append(df)

resultFinal = pd.concat(resultsFinal, axis=0, ignore_index=True)
prepared_file_path = 'prepared_data_all.csv'
resultFinal.to_csv(prepared_file_path, index=False)

df = pd.concat(resultsDF, axis=0, ignore_index=True)
prepared_file_path2 = 'prepared_data_all_part2.csv'
df.to_csv(prepared_file_path2, index=False)